# Job Queue Demo: Qubit Spectroscopy + Amplitude Rabi

This notebook demonstrates the job queue using **CharacterizationRunner**.

**Workflow:**
1. **Qubit Spectroscopy** - Find the qubit frequency
2. **Amplitude Rabi** - Calibrate pulse amplitude using the found frequency

The only difference from local execution is adding `job_client` and `user` to the runner.

## Prerequisites

Start the server and worker in separate terminals:

```bash
# Terminal 1: Start the FastAPI server
cd /Users/conniemiao/GDriveStanford/SchusterLab/local_multimode
python -m uvicorn multimode_expts.job_server.server:app --host 0.0.0.0 --port 8000

# Terminal 2: Start the worker (mock mode for testing)
python -m multimode_expts.job_server.worker --mock
```

## Setup

In [ ]:
import sys
import os

# Add multimode_expts to path for imports
# Adjust this path if running from a different location
sys.path.insert(0, "/Users/conniemiao/GDriveStanford/SchusterLab/local_multimode/multimode_expts")

from copy import deepcopy

import experiments as meas
from slab import AttrDict
from experiments import CharacterizationRunner
from job_server import JobClient
from job_server.mock_hardware import MockStation

# Initialize station (mock for demo)
station = MockStation(experiment_name="job_queue_demo")

# Initialize job client
client = JobClient()

# Check server health
health = client.health_check()
print(f"Server status: {health['status']}")
print(f"Pending jobs: {health['pending_jobs']}")

## Qubit Spectroscopy

In [ ]:
# Defaults
# =================================
gespec_defaults = AttrDict(dict(
    reps=500,
    rounds=1,
    start=4500,  # Will be overwritten by preprocessor
    step=0.1,    # Will be overwritten by preprocessor
    expts=201,
    gain=1000,
    sigma=0.05,
    length=10,
    qubit=[0],
    qubits=[0],
    prepulse=False,
    pre_sweep_pulse=[],
    gate_based=False,
    relax_delay=250,
))

def gespec_preproc(station, default_expt_cfg, **kwargs):
    expt_cfg = deepcopy(default_expt_cfg)

    span = kwargs.pop('span', 20)  # MHz
    center = kwargs.pop('center', station.config_thisrun.device.qubit.f_ge[0])
    expts = kwargs.pop('expts', default_expt_cfg.expts)

    expt_cfg.start = center - span / 2
    expt_cfg.step = span / expts
    
    expt_cfg.update(kwargs)
    return expt_cfg

def gespec_postproc(station, expt):
    # In mock mode, simulate a found frequency
    # Real code would use: new_freq = expt.data['fit_avgi'][2]
    new_freq = 4550.5  # Simulated
    old_freq = station.config_thisrun.device.qubit.f_ge[0]
    station.config_thisrun.device.qubit.f_ge = [new_freq]
    print(f'Updated qubit frequency from {old_freq} to {new_freq}!')

In [ ]:
# Execute
# =================================
gespec_runner = CharacterizationRunner(
    station=station,
    ExptClass=meas.PulseProbeSpectroscopyExperiment,
    default_expt_cfg=gespec_defaults,
    preprocessor=gespec_preproc,
    postprocessor=gespec_postproc,
    job_client=client,  # <-- This enables job queue mode
    user="Claude",
)

expt = gespec_runner.run(center=4550, span=100)

## Amplitude Rabi

In [ ]:
# Defaults
# =================================
amprabi_defaults = AttrDict(dict(
    reps=1000,
    rounds=1,
    start=0,
    step=100,
    expts=50,
    sigma_test=0.035,
    flat_length=0,
    pulse_type='gauss',
    checkEF=False,
    pulse_ge_init=False,
    prepulse=False,
    user_defined_freq=[False, 0],  # Use f_ge from config
    qubits=[0],
    relax_delay=200,
))

def amprabi_preproc(station, default_expt_cfg, **kwargs):
    expt_cfg = deepcopy(default_expt_cfg)
    expt_cfg.update(kwargs)
    return expt_cfg

def amprabi_postproc(station, expt):
    # In mock mode, simulate a found pi-pulse gain
    # Real code would use: pi_gain = expt.data['pi_gain'] or fit result
    pi_gain = 1500  # Simulated
    old_gain = station.config_thisrun.device.qubit.pulses.pi_ge.gain[0]
    station.config_thisrun.device.qubit.pulses.pi_ge.gain = [pi_gain]
    print(f'Updated pi_ge gain from {old_gain} to {pi_gain}!')

In [ ]:
# Execute
# =================================
amprabi_runner = CharacterizationRunner(
    station=station,
    ExptClass=meas.AmplitudeRabiExperiment,
    default_expt_cfg=amprabi_defaults,
    preprocessor=amprabi_preproc,
    postprocessor=amprabi_postproc,
    job_client=client,
    user="Claude",
)

expt = amprabi_runner.run()

## Summary

In [ ]:
print("=" * 60)
print("JOB AND CONFIG IDS")
print("=" * 60)

# Qubit Spectroscopy
spec_result = gespec_runner.last_job_result
print(f"\nQubit Spectroscopy:")
print(f"  Job ID:              {spec_result.job_id}")
print(f"  Hardware Config:     {spec_result.hardware_config_version_id}")
print(f"  Data File:           {spec_result.data_file_path}")

# Amplitude Rabi
rabi_result = amprabi_runner.last_job_result
print(f"\nAmplitude Rabi:")
print(f"  Job ID:              {rabi_result.job_id}")
print(f"  Hardware Config:     {rabi_result.hardware_config_version_id}")
print(f"  Data File:           {rabi_result.data_file_path}")

print("\n" + "=" * 60)
print("CALIBRATION RESULTS")
print("=" * 60)
print(f"Qubit frequency:  {station.config_thisrun.device.qubit.f_ge[0]} MHz")
print(f"Pi-pulse gain:    {station.config_thisrun.device.qubit.pulses.pi_ge.gain[0]}")

## Queue Status

In [ ]:
# Check server health
health = client.health_check()
print(f"Server status:    {health['status']}")
print(f"Pending jobs:     {health['pending_jobs']}")
print(f"Running jobs:     {health['running_jobs']}")

In [ ]:
# View current queue (pending and running jobs)
client.print_queue()

In [ ]:
# View recent job history
print("Recent Job History:")
print("-" * 80)
history = client.get_history(limit=10)
for job in history:
    print(f"{job['job_id']}  {job['status']:10s}  {job['experiment_class']}")

## Config Versioning

The job system tracks versioned snapshots of config files. Key operations:

1. **View available versions** - List all snapshots of a config type
2. **Create a new version** - Snapshot a config file (automatically done by worker, or manually)
3. **Set main config** - Point "main" to a specific version ID
4. **Pull main config** - Get the path to the current main config

In [ ]:
from pathlib import Path
from job_server.database import get_database
from job_server.config_versioning import ConfigVersionManager
from job_server.models import ConfigType

# Initialize database and config manager
db = get_database()
CONFIG_DIR = Path("/Users/conniemiao/GDriveStanford/SchusterLab/local_multimode/multimode_expts/configs")
config_manager = ConfigVersionManager(CONFIG_DIR)

In [ ]:
# 1. List all available config versions
print("Available Hardware Config Versions:")
print("-" * 70)

with db.session() as session:
    versions = config_manager.list_versions(ConfigType.HARDWARE_CONFIG, session)
    for v in versions[:10]:  # Show last 10
        print(f"  {v.version_id}  {v.original_filename}  ({v.created_at})")

In [ ]:
# 2. Create a new config version (snapshot a config file)
# This creates a versioned copy with a unique ID like CFG-HW-20260114-00001

HARDWARE_CONFIG_PATH = CONFIG_DIR / "hardware_config_202505.yml"

with db.session() as session:
    version_id, snapshot_path = config_manager.snapshot_hardware_config(
        source_path=HARDWARE_CONFIG_PATH,
        session=session,
        job_id=None,  # Optional: link to a job
    )
    print(f"Created config version: {version_id}")
    print(f"Snapshot saved to: {snapshot_path}")

In [ ]:
# 3. Set a specific version as the "main" config
# This is useful when you want to switch back to a previous known-good config

# First, let's see what the current main config is (if any)
with db.session() as session:
    main_config = config_manager.get_main_config(ConfigType.HARDWARE_CONFIG, session)
    if main_config:
        print(f"Current main config: {main_config.version_id}")
    else:
        print("No main config set yet")

# Now set a specific version as main
# Replace with an actual version_id from the list above
with db.session() as session:
    # Get the most recent version to use as an example
    versions = config_manager.list_versions(ConfigType.HARDWARE_CONFIG, session)
    if versions:
        example_version_id = versions[0].version_id  # Most recent
        config_manager.set_main_version(
            config_type=ConfigType.HARDWARE_CONFIG,
            version_id=example_version_id,
            session=session,
            updated_by="Claude",
        )
        print(f"Set main config to: {example_version_id}")

In [ ]:
# 4. Get/pull the main config path
# Use this when you need to load the current "canonical" config

with db.session() as session:
    main_path = config_manager.get_main_config_path(ConfigType.HARDWARE_CONFIG, session)
    if main_path:
        print(f"Main config path: {main_path}")
        print(f"File exists: {main_path.exists()}")
    else:
        print("No main config path set")

In [ ]:
# 5. Push: Snapshot a config AND set it as main in one step
# This is the most common operation when you've updated your config and want to publish it

with db.session() as session:
    version_id, snapshot_path = config_manager.push_hardware_config_to_main(
        source_path=HARDWARE_CONFIG_PATH,
        session=session,
        updated_by="Claude",
    )
    print(f"Created and set as main: {version_id}")
    print(f"Snapshot path: {snapshot_path}")

### Using Config Versions with Jobs

When jobs run, the worker automatically snapshots the current configs and records the version IDs with the job. You can see which config versions were used by checking `last_job_result`:

```python
result = runner.last_job_result
print(f"Hardware config used: {result.hardware_config_version_id}")
print(f"Multiphoton config used: {result.multiphoton_config_version_id}")
```

The worker uses the configs from the station's paths at execution time. Config versioning is primarily for:
1. **Reproducibility** - Know exactly which config was used for each job
2. **Rollback** - Switch "main" back to a previous known-good config
3. **Audit trail** - Track config changes over time